In [ ]:
import torch
import torch.nn as nn  # neural network modules
import torch.nn.functional as F  # activation functions
import torch.optim as optim  # optimizer
from torch.autograd import Variable # add gradients to tensors
from torch.nn import Parameter # model parameter functionality
import sklearn 

import torchvision
import torchvision.datasets as datasets
from torchvision.utils import save_image
from torchvision import transforms

import numpy as np
import matplotlib.pyplot as plt
import time
import tqdm

In [ ]:
class FCN(nn.Module):
    def __init__(self, input_dim):
        super(FCN, self).__init__()
        
        self.input_dim = input_dim

        self.layer1 = nn.Linear(input_dim, 4000)
        self.layer2 = nn.Linear(4000, 2000)
        self.layer3 = nn.Linear(2000, 1000)
        self.layer4 = nn.Linear(1000, 500)
        self.layer5 = nn.Linear(500, 250)
        self.layer6 = nn.Linear(250, 100)
        self.layer7 = nn.Linear(100, 50)
        self.layer8 = nn.Linear(50, 20)
        self.layer9 = nn.Linear(20, 1)

    def forward(self, x):
        h1 = F.relu(self.layer1(x))
        h2 = F.relu(self.layer2(h1))
        h3 = F.relu(self.layer3(h2))
        h4 = F.relu(self.layer4(h3))
        h5 = F.relu(self.layer5(h4))
        h6 = F.relu(self.layer6(h5))
        h7 = F.relu(self.layer7(h6))
        h8 = F.relu(self.layer8(h7))
        h9 = F.relu(self.layer9(h8))

        return F.sigmoid(h9)

In [ ]:
### Training function
def train_epoch_classifier(encoder, decoder, classifier, device, dataloader, loss_fn, optimizer):
    # Set train mode for both the encoder and the decoder
    encoder.eval()
    decoder.eval()
    train_loss = []
    # Iterate the dataloader (we do not need the label values, this is unsupervised learning)
    for image_batch, labels in dataloader: # with "_" we just ignore the labels (the second element of the dataloader tuple)
        # Move tensor to the proper device
        image_batch = image_batch.to(device)
        # Encode data
        # print(f"Shape of image batch is {image_batch.shape}")
        encoded_data = encoder(image_batch)
        # Decode data
        decoded_data = decoder(encoded_data)
        output = classifier(decoded_data)
        # Evaluate loss
        loss = loss_fn(output, labels)
        # Backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        # Print batch loss
        train_loss.append(loss.detach().cpu().numpy())

    return np.mean(train_loss)


### Testing function
def test_epoch_classifier(encoder, decoder, classifier, device, dataloader, loss_fn):
    # Set evaluation mode for encoder and decoder
    encoder.eval()
    decoder.eval()
    classifier.eval()
    with torch.no_grad(): # No need to track the gradients
        # Define the lists to store the outputs for each batch
        conc_out = []
        conc_label = []
        for image_batch, test_labels in dataloader:
            # Move tensor to the proper device
            image_batch = image_batch.to(device)
            # Encode data
            encoded_data = encoder(image_batch)
            # Decode data
            decoded_data = decoder(encoded_data)
            output = classifier(decoded_data)
            # Append the network output and the original image to the lists
            
        # Evaluate global loss
        val_loss = loss_fn(output, test_labels)
    return val_loss.data

In [ ]:
num_epochs = 200
diz_loss = {'train_loss':[],'val_loss':[]}
for epoch in range(num_epochs):
   train_loss = train_epoch_classifier(encoder,decoder,device,train_loader,loss_fn,optim)
   val_loss = test_epoch_classifier(encoder,decoder,device,test_loader,loss_fn)
   print('\n EPOCH {}/{} \t train loss {} \t val loss {}'.format(epoch + 1, num_epochs,train_loss,val_loss))
   diz_loss['train_loss'].append(train_loss)
   diz_loss['val_loss'].append(val_loss)

   scheduler.step(val_loss)

torch.save({
            'epoch': epoch,
            'model_state_dict': encoder.state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            }, "./data/most_recent_encoder.h5")

torch.save({
            'epoch': epoch,
            'model_state_dict': decoder.state_dict(),
            'optimizer_state_dict': optim.state_dict(),
            }, "./data/most_recent_decoder.h5")

2048.0

In [6]:
import torch
import torch.nn as nn
data = torch.rand((1, 50,50))
layer = nn.Conv2d(1, 1, kernel_size=4, stride=1, padding=1)
layer(data).shape

torch.Size([1, 49, 49])

In [7]:
data = torch.rand((1, 50,50))
layer = nn.Conv2d(1, 1, kernel_size=8, stride=5, padding=0)
layer(data).shape

torch.Size([1, 9, 9])

In [8]:
data = torch.rand((1, 50,50))
layer = nn.MaxPool2d(kernel_size=10, stride=2, padding=2)
layer(data).shape

torch.Size([1, 23, 23])

In [9]:
data = torch.rand((1, 50,50))
layer = nn.MaxPool2d(kernel_size=2, stride=1, padding=0)
layer(data).shape

torch.Size([1, 49, 49])